In [4]:
import json, re
import pandas as pd
import numpy as np

from sklearn import preprocessing
from math import sqrt
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score as AUC

In [5]:
# Data fields

# TripType - a categorical id representing the type of shopping trip the customer made. This is the ground truth that you are predicting. TripType_999 is an "other" category.
# VisitNumber - an id corresponding to a single trip by a single customer
# Weekday - the weekday of the trip
# Upc - the UPC number of the product purchased
# ScanCount - the number of the given item that was purchased. A negative value indicates a product return.
# DepartmentDescription - a high-level description of the item's department
# FinelineNumber - a more refined category for each of the products, created by Walmart

train = pd.read_csv('train.csv')
test = pd.read_csv("test.csv")

train_reference = train.copy(deep=True)

In [4]:
# MultiColumnLabelEncoder(columns=['fruit', 'color']).fit_transform(fruit_data)

In [ ]:
# numeric x

numeric_cols = ['VisitNumber', 'Upc', 'ScanCount', 'FinelineNumber']
x_num_train = train_data[numeric_cols ].as_matrix()
x_num_test = test_data[numeric_cols ].as_matrix()

# # y

# y_train = train.TripType
# y_test = test.FinelineNumber

# # # # # categorical

# cat_train = train_data.drop(numeric_cols + ['TripType'], axis = 1 )
# cat_test = test_data.drop(numeric_cols, axis = 1 )

# cat_train.fillna(0, inplace = True )
# cat_test.fillna(0, inplace = True )

# x_cat_train = cat_train.T.to_dict().values()
# x_cat_test = cat_test.T.to_dict().values()

# # # # vectorize

# vectorizer = DV(sparse = False)
# vec_x_cat_train = vectorizer.fit_transform(x_cat_train)
# vec_x_cat_test = vectorizer.transform(x_cat_test )

# # # complete x

# x_train = np.hstack((x_num_train, vec_x_cat_train ))
# x_test = np.hstack((x_num_test, vec_x_cat_test ))

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm

In [6]:
# -*- coding: utf-8 -*-
""" Small script that shows hot to do one hot encoding
    of categorical columns in a pandas DataFrame.
    See:
    http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder
    http://scikit-learn.org/dev/modules/generated/sklearn.feature_extraction.DictVectorizer.html
"""
import pandas
import random
import numpy
from sklearn.feature_extraction import DictVectorizer

def one_hot_dataframe(data, cols, replace=False):
    """ Takes a dataframe and a list of columns that need to be encoded.
        Returns a 3-tuple comprising the data, the vectorized data,
        and the fitted vectorizor."""
    vec = DictVectorizer(sparse=False)
    vecData = pd.DataFrame(vec.fit_transform(data[cols].T.to_dict().values()))
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    if replace is True:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
    return (data, vecData, vec)

In [7]:
# Vectorize the categorical columns: Weekday & DepartmentDescription
train_data, _, _ = one_hot_dataframe(train, ['Weekday', 'DepartmentDescription'], replace=True)

In [8]:
test_data, _, _ = one_hot_dataframe(test, ['Weekday', 'DepartmentDescription'], replace = True)

In [11]:
train_as_dicts = [dict(r.iteritems()) for _, r in train_data.iterrows()]#dictionary of keys (columns) with instances (values)
train_features = DictVectorizer(sparse=False).fit_transform(train_as_dicts)

In [32]:
import pandas as pd
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing

# encode labels
labels = train[['TripType']]
le = preprocessing.LabelEncoder()
labels_fea = le.fit_transform(labels) 
# vectorize training data
del train['TripType']
train_as_dicts = [dict(r.iteritems()) for _, r in train.iterrows()]
train_fea = DictVectorizer(sparse=False).fit_transform(train_as_dicts)
# use decision tree
dt = tree.DecisionTreeClassifier()
dt.fit(train_fea, labels_fea)
# transform result
predictions = le.inverse_transform(dt.predict(train_fea).astype('I'))
predictions_as_dataframe = train.join(pd.DataFrame({"Prediction": predictions}))
print predictions_as_dataframe

        VisitNumber Weekday          Upc  ScanCount     DepartmentDescription  \
0                 5  Friday  68113152929         -1        FINANCIAL SERVICES   
1                 7  Friday  60538815980          1                     SHOES   
2                 7  Friday   7410811099          1             PERSONAL CARE   
3                 8  Friday   2238403510          2     PAINT AND ACCESSORIES   
4                 8  Friday   2006613744          2     PAINT AND ACCESSORIES   
5                 8  Friday   2006618783          2     PAINT AND ACCESSORIES   
6                 8  Friday   2006613743          1     PAINT AND ACCESSORIES   
7                 8  Friday   7004802737          1     PAINT AND ACCESSORIES   
8                 8  Friday   2238495318          1     PAINT AND ACCESSORIES   
9                 8  Friday   2238400200         -1     PAINT AND ACCESSORIES   
10                8  Friday   5200010239          1               DSD GROCERY   
11                8  Friday 

/Users/sahluwalia/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [64]:
predictions_as_dataframe.head(3)

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Prediction
0,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000,999
1,7,Friday,60538815980,1,SHOES,8931,30
2,7,Friday,7410811099,1,PERSONAL CARE,4504,30


In [50]:
predictions = predictions_as_dataframe[['Prediction']]

In [53]:
train_reference_triptype = train_reference[['TripType']]

In [56]:
superimposed = predictions.join(train_reference_triptype)

In [65]:
# Create an empty column for the full names
superimposed['Values'] = np.NaN

# Create an iteration counter
i = 0

# For each element in first_name and last_name,
for prediction, true in zip(superimposed['Prediction'], superimposed['TripType']):
    # Change the value of the i'th row in full_name 
    # to the combination of the first and last name
    if superimposed['Prediction'][i] == superimposed['TripType'][i]:
        break

    # Add one to the iteration counter
    i = i+1

In [1]:
def fill_with_bars(old_df, number=10):
    empty_col = [' '*i for i in range(1,number+1)]
    tmp = df(columns=empty_col)
    return pd.concat([tmp,old_df], axis=1).fillna('')

In [9]:
train_data.fillna(0, inplace = True )
test_data.fillna(0, inplace = True )

In [14]:
train_data.to_csv('Train_Vectorizer.csv', encoding='utf-8', index=False)

In [13]:
test_data.to_csv('Test_Vectorizer.csv', encoding='utf-8', index=False)